# Darcy equation

In this tutorial we present how to solve an evolutionary Darcy equation with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unkwons are the velocity $q$ and the pressure $p$.

Let $\Omega=(0,1)^2$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Let $(0,T)$ with $10=T>0$ be the overall simulation period. Given 
$k$ the matrix permeability, we want to solve the following problem: find $({q}, p)$ such that
$$
\left\{
\begin{array}{ll}
\begin{array}{l} 
k^{-1} {q} + \nabla p = {- \rho g \nabla y}\\
p_t + \nabla \cdot {q} = f
\end{array}
&\text{in } \Omega \times (0,T)
\end{array}
\right.
$$
with boundary conditions:
$$ p = 0 \text{ on } \partial_{top} \Omega \times (0,T] \qquad p = \rho g \text{ on } \partial_{bottom} \Omega \times (0,T] \qquad \nu \cdot q = 0 \text{ on } \partial_{left} \Omega \cup \partial_{right} \Omega \times (0,T] $$
and initial conditions:
$$ p|_{t=0} = (1-y) \rho g \text{ in } \Omega \qquad q|_{t=0} = 0 \text{ in } \Omega $$

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [1]:
%load_ext Cython

In [2]:
import shutil
import os

import numpy as np
import scipy.sparse as sps
from scipy.sparse import linalg
import scipy.integrate as integrate
import sympy as sp

import porepy as pp
import pygeon as pg

import time
from math import ceil, floor, log10, exp

import matplotlib.pyplot as plt

/opt/conda/lib/python3.11/site-packages/porepy/numerics/nonlinear/nonlinear_solvers.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange  # type: ignore


Initial parameters definition

In [3]:
N = 10
dt = 0.1

T = 3

output_directory = 'lagrange_multiplier'

In [4]:
# Van Genuchten model parameters ( relative permeability model )
theta_s = 0.396
theta_r = 0.131

alpha = 0.423

n = 2.06
K_s = 4.96e-2
S_s = 0.1

L = 3.5e-3

m = 1 - 1/n

In [5]:
# Relative and absolute tolerances for the non-linear solver
abs_tol = 1e-6
rel_tol = 1e-6

# Order for the quadrature formulae
quad_order = 5

### $\theta$ and $K$

In [6]:
# Symbolic psi
psi_var = sp.Symbol('psi', negative=True)

# Symbolic Theta
theta_expression = theta_r + (theta_s - theta_r) / (1 + (-alpha * psi_var) ** n) ** m
effective_saturation = (theta_expression - theta_r) / (theta_s - theta_r)

# Symbolic Conductivity K
hydraulic_conductivity_expression = K_s * (effective_saturation ** 0.5) * ( 1 - (1 - effective_saturation ** (1 / m)) ** m ) ** 2

In [7]:
# Theta lambda
theta_lambda = sp.lambdify(psi_var, theta_expression, 'numpy')

# Conductivity tensor lambda
conductivity_lambda = sp.lambdify(psi_var, hydraulic_conductivity_expression, 'numpy')

In [8]:
# Actual (and final) theta function
def theta(psi):
    mask = np.where(psi < 0)
    res = np.ones_like(psi) * theta_s
    res[mask] = theta_lambda(psi[mask])

    return res

In [9]:
%%cython

theta_s = 0.396
theta_r = 0.131

alpha = 0.423

n = 2.06
K_s = 4.96e-2

m = 1 - 1/n

def R_effective_saturation(psi):
    return ((theta_r + (theta_s - theta_r) / (1 + (-alpha * psi) ** n) ** m) - theta_r) / (theta_s - theta_r)


def R_K11(x, y, psi):
    if psi >= 0:
        return K_s
    tmp = R_effective_saturation(psi)
    return K_s * (tmp ** 0.5) * ( 1 - (1 - tmp ** (1 / m)) ** m ) ** 2

def R_K12(x, y, psi):
    return 0

def R_K21(x, y, psi):
    return 0

def R_K22(x, y, psi):
    if psi >= 0:
        return K_s
    tmp = R_effective_saturation(psi)
    return K_s * (tmp ** 0.5) * ( 1 - (1 - tmp ** (1 / m)) ** m ) ** 2

### Domain preparation and boundary conditions

In [10]:
top_grid = pp.StructuredTriangleGrid([N, N], [1,1])
top_grid.compute_geometry()

bottom_grid = pp.StructuredTriangleGrid([N, N], [1,1])
bottom_grid.compute_geometry()

In [11]:
boundary_grid, boundary_face_map, boundary_node_map  = pp.partition.extract_subgrid(top_grid, 
                                                                                    top_grid.face_centers[1, :] == np.min(top_grid.face_centers[1, :]), 
                                                                                    faces=True)

With the following code we set the data, in particular the permeability tensor and the boundary conditions. Since we need to identify each side of $\partial \Omega$ we need few steps.

In [12]:
key = "flow"

In [13]:
top_RT0 = pg.RT0(key)
top_P0 = pg.PwConstants(key)
top_data = {}

top_dof_h =  top_P0.ndof( top_grid )
top_dof_q = top_RT0.ndof( top_grid )

In [14]:
bottom_RT0 = pg.RT0(key)
bottom_P0 = pg.PwConstants(key)
bottom_data = {}

bottom_dof_h = bottom_P0.ndof( bottom_grid )
bottom_dof_q = bottom_RT0.ndof( bottom_grid )

In [15]:
gamma_lagrange = pg.PwConstants(key)
gamma_data = {}

gamma_dof = gamma_lagrange.ndof( boundary_grid )

In [16]:
# permeability tensor
pp.initialize_data(bottom_grid, bottom_data, key, { "second_order_tensor": pp.SecondOrderTensor(np.ones(bottom_grid.num_cells)),})
pp.initialize_data(   top_grid,    top_data, key, { "second_order_tensor": pp.SecondOrderTensor(np.ones(top_grid.num_cells)),})

{'discretization_matrices': {'flow': {}},
 'parameters': Data object for physical processes flow
 The keyword "flow" has the following parameters specified: second_order_tensor}

In [17]:
dirichlet_bottom_flag = bottom_grid.face_centers[0, :] == np.max( bottom_grid.face_centers[0,:] )
dirichlet_bottom_value = lambda t: - bottom_RT0.assemble_nat_bc(bottom_grid, lambda x: 1, dirichlet_bottom_flag)

neumann_bottom_flag  = np.hstack((np.array(np.logical_or( bottom_grid.face_centers[0, :] == np.min(bottom_grid.face_centers[0, :]), 
                                                          bottom_grid.face_centers[1, :] == np.min(bottom_grid.face_centers[0, :])  ), dtype=bool), 
                                 np.zeros(shape=bottom_dof_h, dtype=bool)))
neumann_bottom_value = np.zeros(shape=(bottom_dof_h + bottom_dof_q))

In [18]:
dirichlet_bottom_flag.sum(), neumann_bottom_flag.sum()

(10, 20)

In [19]:
dirichlet_top_flag  = top_grid.face_centers[1, :] == np.max(top_grid.face_centers[1,:])
dirichlet_top_value = lambda t: - top_RT0.assemble_nat_bc(top_grid, lambda x: min(1 + t, 2), dirichlet_top_flag)

neumann_top_flag  = np.hstack((np.logical_or( top_grid.face_centers[0, :] == np.min(top_grid.face_centers[0, :]), 
                                             top_grid.face_centers[0, :] == np.max(top_grid.face_centers[0, :]) ), 
                                            np.zeros(shape=top_dof_h, dtype=bool)))
neumann_top_value = np.zeros(shape=(top_dof_h + top_dof_q))

In [20]:
dirichlet_top_flag.sum(), neumann_top_flag.sum()

(10, 20)

In [21]:
neumann_flag = np.hstack( (neumann_bottom_flag, neumann_top_flag, np.zeros(shape=(gamma_dof), dtype=bool)) )
neumann_value = np.hstack( (neumann_bottom_value, neumann_top_value, np.zeros(shape=(gamma_dof))) )

Once the data are assigned to the mixed-dimensional grid, we construct the matrices. In particular, the linear system associated with the equation at every timestep is given as
$$
\left(
\begin{array}{cc} 
M_K &     B^\top\\
-\Delta t B  & M_p
\end{array}
\right)
\left(
\begin{array}{c} 
q^{n+1}\\ 
p^{n+1}
\end{array}
\right)
=\left(
\begin{array}{c} 
p_{\partial} + M_p g\\ 
M_p p^n + \Delta t M_p f^{n+1}
\end{array}
\right)
$$<br>
where $p_{\partial}$ is the vector associated to the pressure boundary contions, $(\cdot)^n$ is a generic variable at the n-th time instant. We can now construct the initial building blocks of the monolithic matrix:

In [22]:
top_mass_h =  top_P0.assemble_mass_matrix(top_grid, top_data)
top_B = - top_mass_h @ pg.div( top_grid )

In [23]:
bottom_mass_h =  bottom_P0.assemble_mass_matrix( bottom_grid, bottom_data )
bottom_mass_q = bottom_RT0.assemble_mass_matrix( bottom_grid, bottom_data ) * K_s
bottom_B = - bottom_mass_h @ pg.div( bottom_grid )

In [24]:
top_proj_q = top_RT0.eval_at_cell_centers( top_grid )
top_proj_h = top_P0.eval_at_cell_centers( top_grid )

bottom_proj_q = bottom_RT0.eval_at_cell_centers( bottom_grid )
bottom_proj_h = bottom_P0.eval_at_cell_centers( bottom_grid )

In [25]:
# assemble initial solution
initial_solution = np.zeros(top_dof_q + top_dof_h + bottom_dof_q + gamma_dof + bottom_dof_h)

In [26]:
bottom_q_mask = np.zeros_like(initial_solution, dtype=bool)
bottom_q_mask[:(bottom_dof_q)] = True

bottom_h_mask = np.zeros_like(initial_solution, dtype=bool)
bottom_h_mask[ (bottom_dof_q) : (bottom_dof_q  + bottom_dof_h) ] = True

top_q_mask = np.zeros_like(initial_solution, dtype=bool)
top_q_mask[(bottom_dof_q + bottom_dof_h):(bottom_dof_q + bottom_dof_h + top_dof_q)] = True

top_h_mask = np.zeros_like(initial_solution, dtype=bool)
top_h_mask[ (bottom_dof_q + bottom_dof_h + top_dof_q) : (bottom_dof_q + bottom_dof_h + top_dof_q + top_dof_h) ] = True

gamma_multipler = np.zeros_like(initial_solution, dtype=bool)
gamma_multipler[-(gamma_dof):] = True

In [27]:
initial_solution[top_h_mask]    =    top_P0.interpolate(   top_grid, lambda x: 1)
initial_solution[bottom_h_mask] = bottom_P0.interpolate(bottom_grid, lambda x: 1)

In [28]:
top_saver    = pp.Exporter(top_grid,    "top_sol", folder_name=output_directory)
bottom_saver = pp.Exporter(bottom_grid, "bottom_sol", folder_name=output_directory)

def save_step(current_sol, step):
    ins = list()

    ins.append((top_grid, "cell_q", ( top_proj_q @ current_sol[top_q_mask] ).reshape((3, -1), order="F")))
    ins.append((top_grid, "cell_h", top_proj_h @ current_sol[top_h_mask]))

    top_saver.write_vtu(ins, time_step=step)

    ins = list()

    ins.append((bottom_grid, "cell_q", ( bottom_proj_q @ current_sol[bottom_q_mask] ).reshape((3, -1), order="F")))
    ins.append((bottom_grid, "cell_h", bottom_proj_h @ current_sol[bottom_h_mask]))

    bottom_saver.write_vtu(ins, time_step=step)

In [29]:
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)

In [30]:
%%cython
import numpy as np

def q1(x: float, y: float):
    return np.array([-x, -y])

def q2(x: float, y: float):
    return np.array([x-1, y])

def q3(x: float, y: float):
    return np.array([-x, 1-y])

def find_ordering(coord: np.array, N):
    lx = np.argmin(coord[0, :])
    rx = np.argmax(coord[0, :])
    mx = np.setdiff1d(np.array([0,1,2]), np.array([lx, rx]))[0]

    # Vertical Alignment
    if np.abs( coord[0, lx] - coord[0, mx] ) < 1 / (2 * N):
        # lx and mx vertical aligned, rx no
        up =   lx if np.argmax(coord[1, np.array([lx, mx])]) == 0 else mx
        down = lx if np.argmin(coord[1, np.array([lx, mx])]) == 0 else mx

        if np.abs( coord[1, up] - coord[1, rx] ) < 1 / (2 * N):
            return [up, down, rx]
        else:
            return [down, rx, up]
    else:
        # rx and mx vertical aligned, lx no
        up =   rx if np.argmax(coord[1, np.array([rx, mx])]) == 0 else mx
        down = rx if np.argmin(coord[1, np.array([rx, mx])]) == 0 else mx

        if np.abs( coord[1, up] - coord[1, lx] ) < 1 / (2 * N):
            return [up, lx, down]
        else:
            return [down, up, lx]

In [31]:
%%cython
import numpy as np


def Richards_K_func(psi: float, K11, K12, K21, K22):
    return lambda x,y: np.array( [[K11(psi,x,y), K12(psi,x,y)],
                                  [K21(psi,x,y), K22(psi,x,y)]] )

In [32]:
def Richards_local_q(coord, sign, psi):
    M = np.zeros(shape=(3,3))

    ordering = find_ordering(coord, boundary_grid.num_nodes-1)
    orientation = [-1, 1, -1] * sign[ordering]

    q_funcs = [q1, q2, q3]

    K_local = Richards_K_func(psi, R_K11, R_K12, R_K21, R_K22)


    for i in range(3):
        for j in range(3):
            integrand = lambda ys,x: np.array([q_funcs[j](x,y).T @ K_local(x, y) @ q_funcs[i](x,y) for y in np.array(ys)])
            inside = lambda xs, n: np.array([integrate.fixed_quad(integrand, 0, 1-x, args=(x,), n=n)[0] for x in np.array(xs)])
            M[ordering[i], ordering[j]] = orientation[j] * orientation[i] * integrate.fixed_quad(inside, 0, 1, n=quad_order, args=(quad_order,))[0]
    
    return M

In [33]:
%%timeit
Richards_local_q(np.array([[0, 1, 0], [0, 0, 1]]), np.array([-1, -1, 1]), 0.75)

2.44 ms ± 87 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [34]:
def mass_q(h_dofs):
    faces, _, sign = sps.find(top_grid.cell_faces)

    _, _, _, _, _, node_coords = pp.map_geometry.map_grid(
            top_grid, top_data.get("deviation_from_plane_tol", 1e-5)
        )
    
    dim = top_grid.dim
    
    node_coords = node_coords[: dim, :]

    top_RT0._compute_cell_face_to_opposite_node(top_grid, top_data)
    cell_face_to_opposite_node = top_data[top_RT0.cell_face_to_opposite_node]
    
    size_A = np.power(top_grid.dim + 1, 2) * top_grid.num_cells
    rows_A = np.empty(size_A, dtype=int)
    cols_A = np.empty(size_A, dtype=int)
    data_A = np.empty(size_A)
    idx_A = 0

    for c in range(top_grid.num_cells):
        # For the current cell retrieve its faces
        loc = slice(top_grid.cell_faces.indptr[c], top_grid.cell_faces.indptr[c + 1])
        faces_loc = faces[loc]
    
        node = np.flip(np.sort(cell_face_to_opposite_node[c, :]))

        coord_loc = node_coords[:, node]

        local = Richards_local_q(coord_loc, sign[loc], h_dofs[c]-1-top_grid.cell_centers[1,c])

        # Save values for Hdiv-mass local matrix in the global structure
        cols = np.concatenate(faces_loc.size * [[faces_loc]])
        loc_idx = slice(idx_A, idx_A + local.size)
        rows_A[loc_idx] = cols.T.ravel()
        cols_A[loc_idx] = cols.ravel()
        data_A[loc_idx] = local.ravel()
        idx_A += local.size

        #print('')
    
    return sps.coo_matrix((data_A, (rows_A, cols_A))).tocsr()

In [35]:
# solve the problem

sol = [initial_solution]

t = 0

save_step(sol[-1], 0)

In [36]:
top_boundary_restrictor = np.zeros(shape=(gamma_dof, top_dof_q))
top_boundary_restrictor[list(range(gamma_dof)), np.where( top_grid.face_centers[1,:] == np.min(top_grid.face_centers[1,:]) )] = 1
top_boundary_restrictor = sps.csr_matrix(top_boundary_restrictor)

In [37]:
bottom_boundary_restrictor = np.zeros(shape=(gamma_dof, bottom_dof_q))
bottom_boundary_restrictor[list(range(gamma_dof)), np.where( bottom_grid.face_centers[1,:] == np.max(bottom_grid.face_centers[1,:]) )] = 1
bottom_boundary_restrictor = sps.csr_matrix(bottom_boundary_restrictor)

In [38]:
# Helper function to project a function evaluated in the cell center to FEM (scalar)
def project_psi_to_fe(to_project):
    return to_project * top_grid.cell_volumes

In [39]:
# Time Loop
for step in range(1, ceil(T/dt) + 1):
    current_time = step * dt
    
    time_rhs = np.zeros_like(sol[-1])
    time_rhs[top_q_mask]    += dirichlet_top_value(current_time)
    time_rhs[bottom_q_mask] += dirichlet_bottom_value(current_time)

    time_rhs[bottom_h_mask] += S_s * bottom_mass_h @ sol[-1][bottom_h_mask]


    prev = sol[-1]
    for k in range(50):    
        rhs = time_rhs.copy()
        rhs[top_h_mask] += top_mass_h @ ( L * prev[top_h_mask] 
                                         + project_psi_to_fe( theta( top_proj_h @ sol[-1][top_h_mask] - 1 - top_grid.cell_centers[1, :] ) ) 
                                         - project_psi_to_fe( theta( top_proj_h @ prev[top_h_mask] - 1 - top_grid.cell_centers[1, :] ) ) )
        

        top_mass_q = mass_q(prev[top_h_mask])

        spp = sps.bmat(
            [[              bottom_mass_q,          bottom_B.T,                    None,         None, -bottom_boundary_restrictor.T],
            [              -dt * bottom_B, S_s * bottom_mass_h,                    None,         None,                          None],
            [                        None,                None,              top_mass_q,      top_B.T,     top_boundary_restrictor.T],
            [                        None,                None,             -dt * top_B, L*top_mass_h,                          None],
            [ -bottom_boundary_restrictor,                None, top_boundary_restrictor,         None,                          None]], format = 'csc')
        
        ls = pg.LinearSystem(spp, rhs)
        ls.flag_ess_bc(neumann_flag, neumann_value)

        current = ls.solve()

        err = np.sqrt( np.sum(np.power(current[top_h_mask] - prev[top_h_mask], 2) + np.power(current[bottom_h_mask] - prev[bottom_h_mask], 2))  )
        base = np.sqrt( np.sum(np.power(prev[top_h_mask], 2) + np.power(prev[bottom_h_mask], 2)) )


        if err < abs_tol + rel_tol * base:
            sol.append( current )
            break
        else:
            prev = current


    print( sol[-1][-gamma_dof:] )

    save_step(sol[-1], step)

[1.04132601 1.0409242  1.04011067 1.03886409 1.03714791 1.03490294
 1.03203255 1.02837136 1.02361197 1.01711221]
[1.08304631 1.08223607 1.08059588 1.07808318 1.07462506 1.07010331
 1.06432482 1.05695888 1.04739028 1.03433309]
[1.12477096 1.12355221 1.12108519 1.11730614 1.11210575 1.10530683
 1.09661981 1.08554862 1.07117025 1.05155502]
[1.16649529 1.16486804 1.16157421 1.15652883 1.14958622 1.14051016
 1.12891465 1.11413823 1.09495013 1.06877689]
[1.20821883 1.20618309 1.20206248 1.19575081 1.18706602 1.17571289
 1.16120895 1.14272741 1.11872967 1.08599853]
[1.24994127 1.24749706 1.2425497  1.23497178 1.22454488 1.21091476
 1.19350249 1.17131592 1.14250866 1.10321979]
[1.29166219 1.28880952 1.28303544 1.27419134 1.2620224  1.24611537
 1.22579488 1.19990343 1.16628684 1.12044045]
[1.33338099 1.3301199  1.32351915 1.31340892 1.29949803 1.28131422
 1.25808566 1.22848951 1.19006382 1.13766026]
[1.37509707 1.37142757 1.36400021 1.35262394 1.33697121 1.31651077
 1.29037433 1.25707372 1.2138